# July 18th, 2018
## Benjamin Adam Catching
## Andino Lab, Summer Rotation Project
## Poliovirus DIPs Project

The goal is that by the middle of the day the images from the second microfluidics test will be processed by the new microfluidics_analysis script and the results analyzed. The aspects to be summarized into two graphs will be  
1) A bargraph of the percentage of droplets containing a cell for each condition (HeLa, 293T, Fast, Slow, Rock) with a value N for each bar.  
2) A bargraph of the percentage of alive cells, again with a N for each condition.

In [36]:
# Import necessary packages for displaying the results
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import skimage
import glob
import pandas as pd
from tools import microfluidics_analysis as micro

%matplotlib inline

In [13]:
# Set the image directory based on if working on notebook in pycharm or not
in_pycharm = True
if in_pycharm:
    bulk_cell_dir = 'data/20180710_Microfluidic_picture/'
else:
    bulk_cell_dir = '../../data/20180710_Microfluidic_picture/'
    
# Define the six directories that contain image files
hs_files = 'HeLa_Slow/*40X*.tif'
hf_files = 'HeLa_Fast/*40X*.tif'
hsr_files = 'HeLa_Slow_Rock/*40X*.tif'
hfr_files = 'HeLa_Fast_Rock/*40X*.tif'
two93f_files = '293T_Fast/*40X*.tif'
two93fr_files = '293T_Fast_Rock/*40X*.tif'

In [17]:
# Read in the HeLa cell files and sort
hela_slow_files = sorted(glob.glob(bulk_cell_dir + hs_files))
hela_slow_rock_files = sorted(glob.glob(bulk_cell_dir + hsr_files))
hela_fast_files = sorted(glob.glob(bulk_cell_dir + hf_files))
hela_fast_rock_files = sorted(glob.glob(bulk_cell_dir + hfr_files))
two93_fast_files = sorted(glob.glob(bulk_cell_dir + two93f_files))
two93_fast_rock_files = sorted(glob.glob(bulk_cell_dir + two93fr_files))

In [55]:
# Combine all file directories
cell_files = [hela_slow_files,
              hela_slow_rock_files,
              hela_fast_files,
              hela_fast_rock_files,
              two93_fast_files,
              two93_fast_rock_files]

In [74]:
# Define the lists that will be turned into a DataFrame
cell, speed, rock, droplet_area, num_cells, num_dead = \
    [], [], [], [], [], []

In [75]:
# Read in the 293T files
for directory in cell_files:
    for i, bright_file in enumerate(directory[0::2]):
    
        # Define the cell type, speed, and rock or not
        if i <= 3:
            cell_type = 'HeLa'
        else:
            cell_type = '293T'
        if i <= 1:
            speed_type = 'Slow'
        else:
            speed_type = 'Fast'
        if i % 2 == 0:
            rock_type = False
        else:
            rock_type = True
    
        # Shift the index and define the bright and gfp images
        index = i * 2
        bright_file = bright_file
        gfp_file = directory[index+1]
    
        # Initialize the droplet object
        temp_droplet_image = micro.BulkDroplet(bright_file, multi_channel=True)
    
        # Define the regions of the droplets
        temp_droplet_labeled, temp_droplet_props = temp_droplet_image.droplet_segment(testing=False)
    
        # Define the cell attributes of the droplets
        droplet_cells_list, gfp_droplets = micro.cell_bright_gfp_thresh(temp_droplet_labeled, 
                                                              temp_droplet_props,
                                                              bright_file,
                                                              gfp_file)
        #print(micro.diff_cells(droplet_cells_list[0])[0].max())
    
        # Define the number of brightfield and gfp cells
        all_cells = [micro.diff_cells(x)[0].max() for x in droplet_cells_list]
        dead_cells = [micro.diff_cells(x)[0].max() for x in gfp_droplets]
    
        # Make sure that all the lists are the same length
        if len(temp_droplet_props) == len(all_cells) == len(dead_cells):
            for j, droplet in enumerate(temp_droplet_props):
                cell.append(cell_type)
                speed.append(speed_type)
                rock.append(rock_type)
                droplet_area.append(droplet.area)
                num_cells.append(all_cells[j])
                num_dead.append(dead_cells[j])
        else:
            print('Set %d has %d droplets, %d cell regions, and %d dead cell regions'
                  % (i, len(temp_droplet_props), len(all_cells), len(dead_cells)))

The file is updated


The file is updated


The file is updated


The file is updated


The file is updated


The file is updated


The file is updated


In [71]:
# Create the pandas dictionary
column_names = ['cell', 'speed', 'rock', 'droplet_area', 'num_cells', 'num_dead']
column_data = zip(column_names, [cell, 
                                 speed, 
                                 rock, 
                                 droplet_area, 
                                 num_cells, 
                                 num_dead])
dataframe_data = dict(column_data)
initial_cell_data = pd.DataFrame(dataframe_data)

In [72]:
set(list(initial_cell_data.cell))

{'29T', 'HeLa'}